In [2]:
%%writefile server.py

from flask import Flask, request, render_template, redirect, session
import cv2
import numpy as np
import datetime

image = None

app = Flask(__name__)


@app.route('/')
def index():
    return render_template("imageprocessing.html", ctx={"title":"뽀토샵"})


@app.route('/upload', methods=["POST"]) # post방식이 올때 업로드하려면
def upload():
    global image # 글로벌 변수 선언 이 함수 밖에서도 사용 가능 글로벌 함수는 많이 사용 안하는게 좋다. 남발하면 추적하기 힘듬
    f = request.files['file1']
    filename = "./static/" + f.filename
    f.save(filename)
    image = cv2.imread(filename) # 이미지 로딩
    cv2.imwrite("./static/result.jpg", image)
    print(image.shape)
#     session["image"] = image

    return redirect("/")


@app.route('/imageprocess')
def imageprocss():
    global image
    method = request.args.get("method")

    
##########################################################################DAY-1    
    
    
    if method == "original" :
        dst = image
        cv2.imwrite('./static/result.jpg', dst)
    
    if method == "gray" :
        dst = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        cv2.imwrite('./static/result.jpg', dst)
    
    if method == "red" :
        red = image[:,:,2]
        dst = np.zeros(image.shape)
        dst[:,:,2] = red
        dst = dst.astype(np.int64)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "green" :
        green = image[:,:,1]
        dst = np.zeros(image.shape)
        dst[:,:,1] = green
        dst = dst.astype(np.int64)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "blue" :
        blue = image[:,:,0]
        dst = np.zeros(image.shape)
        dst[:,:,0] = blue
        dst = dst.astype(np.int64)
        cv2.imwrite('./static/result.jpg', dst)

        
##########################################################################################DAY-2


    if method == "clipping" :
        alpha = int(request.args.get("alpha"))
        alpha /= 100
        beta = int(request.args.get("beta"))
        dst = np.clip(alpha * image + beta, 0, 255).astype('uint8')
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "LUT_invert" :
        table = np.array([255-i for i in range(256)]).astype("uint8")
        dst = cv2.LUT(image, table)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "LUT_binary" :
        binary = int(request.args.get("binary"))
        table = np.array([ 255 if i > binary else 0 for i in range(256)]).astype("uint8")
        dst = cv2.LUT(image, table)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "LUT_gamma" :
        gamma = int(request.args.get("gamma"))
        gamma /= 100
        table = np.array([((i / 255.0) ** gamma) * 255 
                  for i in np.arange(0, 256)]).astype("uint8")
        dst = cv2.LUT(image, table)
        cv2.imwrite('./static/result.jpg', dst)
        
        
        
    if method == "hist" :
        gmin = float(np.min(image))
        gmax = float(np.max(image))
        dst = ((image - gmin) * 255. / (gmax - gmin)).astype("uint8")
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "histeq" :
        dst = np.array(image)
        b = image[:,:,0]
        g = image[:,:,1]
        r = image[:,:,2]
        hist_b = cv2.equalizeHist(b)
        hist_g = cv2.equalizeHist(g)
        hist_r = cv2.equalizeHist(r)
        dst[:,:,0] = hist_b
        dst[:,:,1] = hist_g
        dst[:,:,2] = hist_r
        cv2.imwrite('./static/result.jpg', dst)


        
##############################################################################DAY-3
    
    if method == "emboss" :
        emboss = np.array([[-1, -1, 0],
                          [-1, 0, 1],
                          [0, 1, 1]], np.float32)
        dst = cv2.filter2D(image, -1, emboss, delta=128)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "sharp" :
        sharp = np.array([[-1, -1, -1],
                          [-1, 9, -1],
                          [-1, -1, -1]], np.float32)
        dst = cv2.filter2D(image, -1, sharp)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "blur" :
        size = int(request.args.get("size"))
        dst = cv2.blur(image, (size, size))
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "gaussian" :
        gsize = int(request.args.get("gsize"))
        sig = int(request.args.get("sig"))
        dst = cv2.GaussianBlur(image, (gsize,gsize), sig)
        cv2.imwrite('./static/result.jpg', dst)

    if method == "filter" :
        mask = np.array([[request.args.get("a1"), request.args.get("a2"), request.args.get("a3")],
                         [request.args.get("a4"), request.args.get("a5"), request.args.get("a6")],
                         [request.args.get("a7"), request.args.get("a8"), request.args.get("a9")]], np.float32)
        
        dst = cv2.filter2D(image, -1, mask)
        cv2.imwrite('./static/result.jpg', dst)
        
    if method == "noise" :
        noise = int(request.args.get("noise"))
        mask = np.zeros(image.shape, np.int32)
        cv2.randn(mask, 0, noise)
        dst = cv2.add(image, mask, dtype=cv2.CV_8UC1)
        cv2.imwrite('./static/result.jpg', dst)
    
        
##############################################################################DAY-4
        
    if method == "rotation" :
        degree = int(request.args.get("degree"))
        cp = (image.shape[1]/2, image.shape[0]/2)
        affine_mat = cv2.getRotationMatrix2D(cp, degree, 1)
    
        dst = cv2.warpAffine(image, affine_mat, (0,0))
        cv2.imwrite('./static/result.jpg', dst)
        
#############################################################################DAY-5



################################################################################Computer Vision
###############################################################################DAY-1

    if method == "lane":
        src = image.copy()
        img_blur = cv2.GaussianBlur(src, (11, 11), 0)
        img_edge = cv2.Canny(img_blur, 60, 150)
        lines = cv2.HoughLinesP(img_edge, 1, 
                                np.pi / 180, 10, 
                                minLineLength=200, 
                                maxLineGap=5)

        leftlane = []
        rightlane = []

        for i in range(lines.shape[0]):
            pt1 = (lines[i][0][0], lines[i][0][1])
            pt2 = (lines[i][0][2], lines[i][0][3])
            w = (pt2[1] - pt1[1]) / (pt2[0]-pt1[0])
            if w < 0 :
                leftlane.append(pt1)
                leftlane.append(pt2)
            else :
                rightlane.append(pt1)
                rightlane.append(pt2)
    
        cv2.line(src, max(leftlane), min(leftlane), (0,255,255), 15, cv2.LINE_AA)
        cv2.line(src, max(rightlane), min(rightlane), (255,255,0), 15, cv2.LINE_AA)

        poly = []
        poly.append(max(leftlane))
        poly.append(min(leftlane))
        poly.append(max(rightlane))
        poly.append(min(rightlane))

        poly = np.array(poly)
        cv2.fillPoly(src,[poly],(255,0,255))
        
        dst = cv2.addWeighted(image, 0.7, src, 0.3, 0)
        cv2.imwrite('./static/result.jpg', dst)

        
    return "hello~~"


if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py
